In [1]:
from pymongo import MongoClient
import json
import glob
import os

In [2]:
# Function to import data from a JSON file
def import_weather_data(file_path, schema, collection, city):
    with open(file_path) as file:
        data = json.load(file)
        for item in data:
            item['city'] = city
            collection.insert_one(item)

In [3]:
# Function to import data from a JSON file
def import_futures_data(file_path, schema, collection):
    with open(file_path) as file:
        data = json.load(file)
        for item in data:
            collection.insert_one(item)

In [4]:
# Establish a connection to your MongoDB server
client = MongoClient('mongodb://localhost:27017/')
db = client['project_3_db']  
weather_collection = db['weather_data']
futures_collection = db['futures_data']


In [5]:
# Define the schema for the weather data
weather_schema = {
    'date': 'date',
    'tavg_fahrenheit': 'double',
    'tmin_fahrenheit': 'double',
    'tmax_fahrenheit': 'double'
}

# Define the schema for the futures data
futures_schema = {
    'Date': 'date',
    'Open': 'double',
    'High': 'double',
    'Low': 'double',
    'Close': 'double',
    'Adj_Close': 'double',
    'Volume': 'int',
    'ATR': 'double'
}

In [6]:
# Specify the directory path where the weather JSON files are located
weather_directory = r'C:\Users\abrid\OneDrive\Desktop\Project 3 Folder\Group-Project-3\data\weather'

# Specify the directory path where the futures JSON files are located
futures_directory = r'C:\Users\abrid\OneDrive\Desktop\Project 3 Folder\Group-Project-3\data\futures'

In [8]:
# Import weather data from multiple files for each city
cities = ['Los Angeles', 'New York City', 'Chicago', 'Detroit', 'Columbus', 'Philadelphia', 'Newark', 'Houston', 'Indianapolis', 'Milwaukee']

for city in cities:
    collection = db['weather_data']  # Use the same collection for all cities
    city_file_path = os.path.join(weather_directory, f'{city}_data.json')
    import_weather_data(city_file_path, weather_schema, collection, city)

In [9]:
# Import futures data from multiple files
futures_files = os.listdir(futures_directory)
for file in futures_files:
    if file.endswith('.json'):
        file_path = os.path.join(futures_directory, file)
        import_futures_data(file_path, futures_schema, futures_collection)

In [10]:
# Close the MongoDB connection
client.close()